In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from bokeh.plotting import output_notebook, show
output_notebook()

Loading BokehJS ...

In [7]:
base = pd.read_csv("HR_comma_sep.csv") #soruce = https://www.kaggle.com/ludobenistant/hr-analytics-1/data

###### We'll use the same file to simulate the following datasets
Train: the data we use to train and validate the models<br>
holdout_data: the final test set to judge the real world model <br>
new_data: new data the model will be applied on in production.

In [3]:
train = base.sample(10000)
holdout_data = base[~base.index.isin(train.index)]
new_data = holdout_data.sample(2000).drop("left", axis=1) # removing the target as this won't have it in real use cases
holdout_data = holdout_data[~holdout_data.index.isin(new_data.index)]

In [4]:
train.shape
holdout_data.shape
new_data.shape

(10000, 10)

(2999, 10)

(2000, 9)

In [5]:
import Model_selection, importlib

###### The following are the parameters used during the class instantiation
modelBase: the training data<br>

target: the target class to predict - must be an integer<br>

scoring: any one of scikit learn's scorers - http://scikit-learn.org/stable/modules/model_evaluation.html<br>

n_jobs: number of CPU cores to use<br>

learning_curve: if true, scikit learn's learning curve will be used to determine the ideal training set size for cross validation<br>

automated: if false, we will have to call functions apply_models and evaluate_models to get the performance characteristics<br>

models: the models we want to test. we can use any or all of gbm, svm, random forest, logistic regression and naive_bayes<br>

categorical_columns: a list of columns that will be used as categorical. If the columns are integer but need to be used as a category, python may change the type at times, so the data will be padded with an underscore 

In [11]:
ms = Model_selection.model_train(modelBase=train, target="left", scoring="neg_log_loss", n_jobs=-1, learning_curve=True, automated=True, models=["gbm", "random_forest", "svm", "logistic_regression"], categorical_columns=["Work_accident", "promotion_last_5years"])

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 306 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   16.1s finished


gbm best CV score: -0.10486186001
gbm fitting complete

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 306 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   10.7s finished


random_forest best CV score: -0.173279428163
random_forest fitting complete

Fitting 5 folds for each of 4 candidates, totalling 20 fits


...

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   31.1s finished


svm best CV score: -0.119445805947
svm fitting complete

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Done  12 out of  25 | elapsed:    0.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.5s finished


logistic_regression best CV score: -0.427050265553
logistic_regression fitting complete

gbm Training accuracy:98.96%
gbm Test accuracy:97.6%
gbm Training f1_score:97.78%
gbm Test f1_score:95.15%

random_forest Training accuracy:94.49%
random_forest Test accuracy:94.9%
random_forest Training f1_score:88.51%
random_forest Test f1_score:90.04%

svm Training accuracy:99.84%
svm Test accuracy:97.3%
svm Training f1_score:99.66%
svm Test f1_score:94.59%

logistic_regression Training accuracy:79.22%
logistic_regression Test accuracy:77.85%
logistic_regression Training f1_score:43.51%
logistic_regression Test f1_score:43.99%



###### Selecting and retraining the model
One or more than one models can be selected and they will be trained on the full train/validation split used above to test on the final holdout set<br>

Multiple holdout sets can be added to the function test_on_holdout_data and the performance plots and tables will bet updated accordingly<br>

The plots will be a part of the dictionary ms.plots<br>
The tables used to calculate the KS and Gini Coeficients will be a part of the dictionary ms.Gini_table

In [35]:
ms.select_retrain_model(["random_forest"])

In [36]:
ms.test_on_holdout_data(X=holdout_data.loc[:, holdout_data.columns!=ms.target], y = holdout_data.loc[:, ms.target])

Re-fitting on the full data...


###### Saving the final model objects 
The following will be saved as pickle objects. They can be loaded and used on new data using the model_apply method<br>

DictVectorizer.pkl: the dictvectorizer object<br>
feature_levels.pkl: the levels of all object variables<br>
Categorical_columns.pkl: all columns specified as categorical <br>
model.pkl: the final model to be applied to the test set (name will vary based on the model)

In [37]:
ms.save_model_objects()

###### Classifying new data
Make sure that the objects are in the relevant folder or that the paths are provided correcty. The model can predict the class probabilies as well as the labels

In [40]:
predictions_probabilities = Model_selection.model_apply(new_data, model="random_forest.pkl", predict_proba=True)
predictions = Model_selection.model_apply(new_data, model="random_forest.pkl", predict_proba=False)

In [41]:
predictions_probabilities
predictions

array([[ 0.99104889,  0.00895111],
       [ 0.85469338,  0.14530662],
       [ 0.23716275,  0.76283725],
       ..., 
       [ 0.99104889,  0.00895111],
       [ 0.93131162,  0.06868838],
       [ 0.99104889,  0.00895111]])

array([0, 0, 1, ..., 0, 0, 0])

###### Performance characteristics saved inside the class

In [18]:
ms.Gini_table["random_forest_test"] # the validation base

,Probability_of_Event,Event,Total,Non_event,Cumulative_Non_event,Cumulative_Event,Population_%,Cumulative_Non_event_%,Cumulative_Event_%,Difference,Event_rate,Gini
9,"(0.884, 1.0]",183,187,4,4,183,0.0935,0.002665,0.366733,0.364069,0.978610,0.000489
8,"(0.754, 0.884]",170,194,24,28,353,0.0970,0.018654,0.707415,0.688761,0.876289,0.008587
7,"(0.118, 0.754]",120,250,130,158,473,0.1250,0.105263,0.947896,0.842633,0.480000,0.071682
6,"(0.0712, 0.118]",3,133,130,288,476,0.0665,0.191872,0.953908,0.762036,0.022556,0.082357
5,"(0.0277, 0.0712]",13,225,212,500,489,0.1125,0.333111,0.979960,0.646849,0.057778,0.136569
4,"(0.00988, 0.0277]",7,209,202,702,496,0.1045,0.467688,0.993988,0.526300,0.033493,0.132824
3,"(0.00909, 0.00988]",1,193,192,894,497,0.0965,0.595603,0.995992,0.400389,0.005181,0.127274
2,"(0.00903, 0.00909]",1,197,196,1090,498,0.0985,0.726183,0.997996,0.271813,0.005076,0.130187
1,"(0.00888, 0.00903]",0,205,205,1295,498,0.1025,0.862758,0.997996,0.135238,0.000000,0.136302
0,"(-0.001, 0.00888]",1,207,206,1501,499,0.1035,1.000000,1.000000,0.000000,0.004831,0.137104


In [21]:
ms.Gini_table["random_forest_test1"] # the first holdout test base

,Probability_of_Event,Event,Total,Non_event,Cumulative_Non_event,Cumulative_Event,Population_%,Cumulative_Non_event_%,Cumulative_Event_%,Difference,Event_rate,Gini
9,"(0.884, 1.0]",318,331.0,13.0,13.0,318,0.110370,0.005679,0.447887,0.442208,0.960725,0.001272
8,"(0.754, 0.884]",247,294.0,47.0,60.0,565,0.098033,0.026212,0.795775,0.769562,0.840136,0.012768
7,"(0.118, 0.754]",93,302.0,209.0,269.0,658,0.100700,0.117519,0.926761,0.809242,0.307947,0.078639
6,"(0.0712, 0.118]",1,16.0,15.0,284.0,659,0.005335,0.124072,0.928169,0.804097,0.062500,0.006078
5,"(0.0277, 0.0712]",38,579.0,541.0,825.0,697,0.193064,0.360419,0.981690,0.621271,0.065630,0.225695
4,"(0.00988, 0.0277]",6,286.0,280.0,1105.0,703,0.095365,0.482744,0.990141,0.507397,0.020979,0.120601
3,"(0.00909, 0.00988]",2,221.0,219.0,1324.0,705,0.073691,0.578419,0.992958,0.414539,0.009050,0.094866
1,"(0.00888, 0.00903]",5,817.0,812.0,2136.0,710,0.272424,0.933159,1.000000,0.066841,0.006120,0.353491
0,"(-0.001, 0.00888]",0,153.0,153.0,2289.0,710,0.051017,1.000000,1.000000,0.000000,0.000000,0.066841
10,All,710,2999.0,2289.0,2289.0,710,1.000000,1.000000,1.000000,0.000000,0.236746,NaN


##### Lorenz curves

In [22]:
show(ms.plots["random_forest"])

###### KS/Gini Coefficients

In [31]:
print(f"Random Forest training Gini:{ms.Gini['random_forest_train']:{4}.{4}}%")
print(f"Random Forest test Gini:{ms.Gini['random_forest_test']:{4}.{4}}%")
print(f"Random Forest holdout sample Gini:{ms.Gini['random_forest_test1']:{4}.{4}}%")

Random Forest training Gini:91.93%
Random Forest test Gini:92.68%
Random Forest holdout sample Gini:92.05%


In [32]:
print(f"Random Forest training KS:{ms.KS['random_forest_train']:{4}.{4}}%")
print(f"Random Forest test KS:{ms.KS['random_forest_test']:{4}.{4}}%")
print(f"Random Forest holdout sample KS:{ms.KS['random_forest_test1']:{4}.{4}}%")

Random Forest training KS:81.93%
Random Forest test KS:84.26%
Random Forest holdout sample KS:80.92%
